In [25]:
# imports
import requests 
import pandas as pd 
import json 
import numpy 
import os 
from IPython.display import JSON
import sqlite3
from sqlite3 import Error



# Foursquare

Send a request to Foursquare with a small radius (1000m) with the location of your choice

In [26]:

# FSQ API keys 

FSQ_key = os.environ["FourSquareAPI"]
#url = "https://api.foursquare.com/v3/places/search?ll=45.43011966884903%2C-75.69100998550019&radius=1000&limit=50&categories=13065"
FSQ_url ="https://api.foursquare.com/v3/places/search"

FSQ_headers = {
    "Accept": "application/json",
    "Authorization": FSQ_key 
}


Parse through the response to get the POI details you want (rating, name, location, etc)

In [27]:
# Setting wanted parameters to fetch from the request 
f_param = {
    'll': '45.429999200432945,-75.69109581618784',
    'radius': '1000',
    'categories': '13065',
    'fields': 'name,tel,location,categories,website,hours,rating',
    'limit': '50'
}


# Data Request
FSQ_Data = requests.get(FSQ_url, params = f_param, headers=FSQ_headers)

In [28]:
# checkthe request was successful 
FSQ_Data.status_code

200

In [20]:
# Export the information that was returned using method .json()
FSQ_Data_Json = FSQ_Data.json()
type(FSQ_Data_Json)

dict

In [21]:
# Using JSON IPython.display to take a clearer look at nested Json Data 
JSON(FSQ_Data_Json)

<IPython.core.display.JSON object>

Put your parsed results into a DataFrame

In [22]:
#Putting Data into pandas Dataframe and flattning out using normalize json for results 

df_f0 = pd.json_normalize(FSQ_Data_Json, record_path='results', errors='ignore')
FSQ_dF = df_f0[['name', 'rating', 'hours.display', 'location.address', 'tel', 'website']].copy()
FSQ_dF.rename(columns={'name': 'Name','hours.display': 'Hours', 'location.address': 'Address', 'tel': 'Phone_number'}, inplace=True)  
FSQ_dF

#FSQ_dF.sort_values(['rating'], ascending=False).head(10)


,Name,rating,Hours,Address,Phone_number,website
0,Chez Lucien,8.7,"Open Daily 00:00-2:00, 11:00-23:59",137 Murray St,(613) 241-3533,http://chezlucien.ca
1,El Camino,8.7,Mon-Wed 16:00-23:59; Thu-Sun 12:00-23:59,81 Clarence St,(613) 422-5918,http://eatelcamino.com
2,The Keg Steakhouse + Bar,8.4,Open Daily 12:00-23:59,75 York St,(613) 241-8514,https://www.kegsteakhouse.com/
3,Bite Burger House,8.4,"Mon-Fri 11:30-23:59; Sat-Sun 00:00-1:00, 11:30...",108 Murray St,(613) 562-2483,http://www.biteburgerhouse.com
4,BeaverTails Canada Inc,8.8,Mon-Wed 10:00-22:00; Thu 10:00-23:00; Fri-Sat ...,69 George St,(613) 241-1230,http://www.beavertailsinc.com/
5,Ahora Mexican Cuisine,8.0,Open Daily 11:30-22:00,307 Dalhousie St,(613) 562-2081,http://www.ahora.ca/
6,Corazon de Maiz,8.5,Open Daily 11:00-18:00,55 by Ward Market Sq,(613) 244-1661,http://www.corazondemaizottawa.com
7,Fiazza Fresh Fired,8.2,Mon-Thu 11:00-21:00; Fri-Sat 11:00-22:00; Sun ...,86 Murray St,(613) 562-2000,http://www.fiazza.ca
8,E18hteen,8.6,Open Daily 17:00-22:30,18 York St,(613) 244-1188,http://www.restaurant18.com
9,JOEY Rideau,8.6,"Mon-Fri 11:00-23:59; Sat-Sun 00:00-1:00, 11:00...",50 Rideau St,(613) 680-5639,https://joeyrestaurants.com/location/joey-ride...


# Yelp

Send a request with the same location paramaters (location, radius, etc)

In [6]:
#  YELP API
YELP_key = os.environ['YELPAPIKEY']
YELP_header = {'Authorization': 'Bearer %s' %YELP_key}
YELP_Url = 'https://api.yelp.com/v3/businesses/search'

In [8]:
# Setting wanted parameters to fetch from the request 
Y_param = {
    'location': '45.429999200432945,-75.69109581618784',
    'radius': '10000',
    'categories': 'restaurants',
    'limit': '50'
}


# Data Requests

YELP_Data = requests.get(YELP_Url, params = Y_param, headers=YELP_header)

Parse through your result and get POI details

In [11]:
# checkthe request was successful 
YELP_Data.status_code

200

In [13]:
# Export the information that was returned using method .json()
YELP_Data_Json = YELP_Data.json()
type(YELP_Data_Json)

dict

In [14]:
# Using JSON IPython.display to take a clearer look at nested Json Data 
JSON(YELP_Data_Json)

<IPython.core.display.JSON object>

Put your parsed results into a DataFrame

In [15]:
#Putting Data into pandas Dataframe and flattning out using normalize json for results
df_y0 = pd.json_normalize(YELP_Data_Json, record_path='businesses', errors='ignore')
YELP_dF= df_y0[['name', 'rating','location.address1', 'display_phone', 'review_count']].copy()
YELP_dF.rename(columns={'name': 'Name','hours.display': 'Hours', 'location.address1': 'Address', 'display_phone': 'Phone_number'}, inplace=True)  
YELP_dF

#YELP_dF.sort_values(['rating'], ascending=False).head(10)

,Name,rating,Address,Phone_number,review_count
0,BeaverTails,4.5,69 George Street,+1 613-241-1230,252
1,La Bottega Nicastro Fine Food Shop,4.5,64 George St,+1 613-789-7575,161
2,Play Food And Wine,4.0,1 York Street,+1 613-667-9207,225
3,The SconeWitch,4.0,150 Elgin Street,+1 613-232-2173,213
4,Shawarma Palace,4.0,464 Rideau Street,+1 613-789-9533,235
5,Sansotei Ramen,4.5,153 Bank Street,+1 613-695-1718,323
6,Wilf & Ada's,4.5,510 Bank Street,+1 613-231-7959,355
7,Benny's Bistro,4.5,119 Murray St,+1 613-789-7941,115
8,Fiazza Fresh Fired,4.0,86 Murray Street,+1 613-562-2000,140
9,Riviera,4.5,62 Sparks Street,+1 613-233-6262,103


In [35]:

# For Details on each business I have to run seperate API calls on each id, its how the api requests are setup(from what I have researched) 

# YELP_id = []
# for y in YELP_Data_Json['businesses']:
#     YELP_id.append(y['id'])
    
# #print(len(y_id))
# # print(YELP_id)

# for i in YELP_id:
#     YELP_Detail_URL = f'https://api.yelp.com/v3/businesses/{i}'
#     YELP_Detail = requests.get(YELP_Detail_URL, headers=YELP_header).json()
    
# print(YELP_Detail)

# # df =  YELP_Detail
# # df_y1 = df[['name', 'rating', 'price','location.address1', 'display_phone', 'review_count']].copy()
# # df_y1.rename(columns={'location.address1': 'address', 'display_phone': 'phone_number'}, inplace=True)
# # df_y1.sort_values(by=['rating', 'review_count'], ascending=False).head(10)


In [36]:
# Using JSON IPython.display to take a clearer look at nested Json Data 
JSON(YELP_Detail)

<IPython.core.display.JSON object>

# Google (stretch)


Use the same process as the first two APIs

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

In [54]:
# Creating a database from the Dataframes using SQLite3

def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")
    return connection

FSQ_Connection = create_connection('Byward_Market.db')

FSQ_db = FSQ_dF.astype({'Name': 'str', 'rating': 'str','Address': 'str', 'Phone_number': 'str', 'website': 'str'}).copy()
FSQ_db.to_sql('FSQ Retaurants', con=FSQ_Connection)
  

YELP_db =YELP_dF.astype({'Name': 'str', 'rating': 'str','Address': 'str', 'Phone_number': 'str', 'review_count': 'str'}).copy()
YELP_db.to_sql('YELP Retaurants', con=FSQ_Connection)



Connection to SQLite DB successful


Get the top 10 restaurants according to their rating

In [23]:
# Top 10 

FSQ_Sort = (FSQ_dF.sort_values(['rating'], ascending=False))
FSQ_Top_10 = FSQ_Sort.loc[ : ,['Name','rating','Address']].head(10)
FSQ_Top_10

,Name,rating,Address
4,BeaverTails Canada Inc,8.8,69 George St
0,Chez Lucien,8.7,137 Murray St
1,El Camino,8.7,81 Clarence St
18,Riviera,8.6,62 Sparks St
8,E18hteen,8.6,18 York St
9,JOEY Rideau,8.6,50 Rideau St
10,Play Food & Wine,8.6,1 York St
6,Corazon de Maiz,8.5,55 by Ward Market Sq
2,The Keg Steakhouse + Bar,8.4,75 York St
3,Bite Burger House,8.4,108 Murray St


In [24]:
YELP_Sort = (YELP_dF.sort_values(['rating'], ascending=False))
YELP_Top_10 = YELP_Sort.loc[:, ['Name','rating','Address']].head(10)
YELP_Top_10

,Name,rating,Address
0,BeaverTails,4.5,69 George Street
40,Fraser Cafe,4.5,7 Springfield Road
26,Stella Luna Gelato Cafe,4.5,1103 Bank St
1,La Bottega Nicastro Fine Food Shop,4.5,64 George St
24,Level One Game Pub,4.5,14 Waller Street
30,Pili Pili,4.5,205 Dalhousie Street
34,Oz Kafe,4.5,10 York St
18,Food Mood: Korean & Japanese Kitchen,4.5,178 Rideau Street
17,Shanghai Wonton Noodle,4.5,178 Rideau Street
39,Bread & Sons Bakery,4.5,195 Bank Street


# Travelling Salesman Problem (stretch)

If you have time, follow the steps in the [ortools tutorial](https://developers.google.com/optimization/routing/tsp) using Google's [Directions API](https://developers.google.com/maps/documentation/directions/start).

In [30]:
print("x")

x
